<a href="https://colab.research.google.com/github/360abhi/FirebaseAngularApp/blob/main/Aprhodite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
!python -m spacy download en_core_web_lg
from spacy.util import filter_spans
from spacy.tokens import DocBin
import spacy
from spacy import displacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
from tqdm import tqdm
nlp = spacy.blank("en")
doc_bin = DocBin()

In [4]:
import pandas as pd
def process_excel_to_format(excel_file):
    df = pd.read_excel(excel_file,sheet_name="Train_Data", engine='openpyxl')
    train_data = []
    for index, row in df.iterrows():

        text = row['Sentences'].lower()

        action = row['ACTION'].lower() if pd.notnull(row['ACTION']) else ""

        page = row['PAGE'].lower() if pd.notnull(row['PAGE']) else ""

        field = row['FIELD'].lower() if pd.notnull(row['FIELD']) else ""

        data = row['DATA'].lower() if pd.notnull(row['DATA']) else ""

        entities = [

            (text.find(action),  text.find(action) + len(action),  "ACTION"),

            (text.find(page),text.find(page) + len(page), "PAGE"),

            (text.find(field),text.find(field) + len(field),"FIELD"),

            (text.find(data),text.find(data) + len(data),  "DATA")

        ]

        train_data.append({"text": text, "entities": entities})

    return train_data

excel_file = "CIM_TestCases.xlsx"

formatted_data = process_excel_to_format(excel_file)

print(formatted_data)


[{'text': 'login into arx with operational maker credential', 'entities': [(0, 5, 'ACTION'), (11, 14, 'PAGE'), (0, 0, 'FIELD'), (20, 48, 'DATA')]}, {'text': 'click on cim', 'entities': [(0, 5, 'ACTION'), (0, 0, 'PAGE'), (9, 12, 'FIELD'), (0, 0, 'DATA')]}, {'text': 'in package tab, click on create package', 'entities': [(16, 21, 'ACTION'), (3, 14, 'PAGE'), (25, 39, 'FIELD'), (0, 0, 'DATA')]}, {'text': 'in details, enter all mandatory details and select customization as allowed, digital channel as allowed, payment limits as valid limit and click on next button', 'entities': [(12, 17, 'ACTION'), (3, 10, 'PAGE'), (0, 0, 'FIELD'), (22, 39, 'DATA')]}, {'text': 'in products page, select\xa0 all products', 'entities': [(18, 24, 'ACTION'), (3, 11, 'PAGE'), (3, 11, 'FIELD'), (3, 11, 'DATA')]}, {'text': 'select appropriate charges', 'entities': [(0, 6, 'ACTION'), (-1, 7, 'PAGE'), (19, 26, 'FIELD'), (0, 0, 'DATA')]}, {'text': 'remove the product level limit for max no of transaction amount per day

In [5]:
train_data = formatted_data

In [6]:
for training_example in tqdm(train_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label,alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
doc_bin.to_disk("./train.spacy")

100%|██████████| 56/56 [00:00<00:00, 943.19it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

In [7]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     57.72   16.14   10.06   40.91    0.16


In [ ]:

nlpner = spacy.load("./model-best")
doc = nlpner("login into arx with maker")
colors = {"PAGE": "#F67DE3", "ACTION": "#7DF6D9", "FIELD": "#a6e22d"}
options = {"ents": ["PAGE", "ACTION", "FIELD"], "colors": colors}
# # Need to pass a list of dictionaries where each dictionary represents one sentence.
data = [{"text": doc.text, "ents": [{"start": ent.start_char, "end": ent.end_char, "label": ent.label_} for ent in doc.ents]}]
displacy.render(data, style="ent", options=options, manual=True, jupyter=True)
print([(ent.text, ent.label_) for ent in doc.ents])